# Preprocessing

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import utm
import random

In [2]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc

In [3]:
# Load jupyter extension to reload packages before executing user code.
# https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
# Reload all packages (except those excluded by %aimport) every time before executing the Python code typed.
%autoreload 2

## Mineralogy

**To do**
* ~~Clean last points in Excel file while using "sum" as check~~

In [4]:
mineralogy = pd.read_excel("../_CLEANED/Vistelius_data_cleaned_all.xlsx", index_col=0)

### Check for wrong entries

In [5]:
np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"])

array([ True,  True,  True, ...,  True,  True,  True])

In [6]:
wrong_sum = mineralogy.loc[~np.isclose(mineralogy.loc[:, :"oth"].sum(axis=1), mineralogy.loc[:, "sum"]), :]

In [7]:
# Check to see if any remaining incorrect lines are present
wrong_sum

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5,l.i.,oth,sum,hs


In [8]:
mineralogy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659 entries, 1 to 4659
Data columns (total 15 columns):
SiO2     4659 non-null float64
TiO2     4626 non-null float64
Al2O3    4659 non-null float64
Fe2O3    4657 non-null float64
FeO      4659 non-null float64
MnO      4544 non-null float64
MgO      4658 non-null float64
CaO      4659 non-null float64
Na2O     4659 non-null float64
K2O      4659 non-null float64
P2O5     3834 non-null float64
l.i.     4659 non-null float64
oth      874 non-null float64
sum      4659 non-null float64
hs       2240 non-null float64
dtypes: float64(15)
memory usage: 582.4 KB


In [9]:
wrong_sum.loc[:, :"oth"].sum(axis=1)

Series([], dtype: float64)

In [10]:
mineralogy = preproc.replace_nan(mineralogy, 0)

In [11]:
mineralogy['oth'] = mineralogy['P2O5'] + mineralogy['l.i.'] + mineralogy['oth']
mineralogy.drop(["l.i.", "P2O5", ], axis = 1, inplace = True)

In [12]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,oth,sum,hs
1,80.80,0.04,10.16,0.61,1.72,0.00,0.40,0.55,2.00,3.59,0.35,100.22,0.00
2,80.00,0.10,10.10,0.17,0.56,0.02,0.40,0.35,2.30,5.10,0.55,99.65,0.00
3,79.92,0.05,9.89,0.16,1.73,0.02,0.12,0.14,0.75,6.15,1.10,100.03,0.30
4,79.65,0.04,9.64,1.15,0.75,0.10,0.45,0.67,3.71,4.25,0.26,100.67,0.01
5,79.18,0.08,10.24,0.64,2.60,0.04,0.05,1.25,1.52,3.08,1.92,100.60,0.28


### Cleaning
**To do**
* ~~Replace zero values~~


In [13]:
# Would not do this this way since it becomes less clear what the variable means
# You should also replace 'minralogy' in all remaining cells by 'x' if you would want to do this
# x = mineralogy

In [14]:
# Replace zero values
mineralogy = preproc.replace_zero(mineralogy, 0.01)

In [15]:
mineralogy.to_excel("../_CLEANED/Vistelius_data_cleaned.xlsx")

* ~~Normalize~~

In [16]:
# Normalize specific columns
mineralogy.loc[:, :"oth"] = preproc.normalize(mineralogy.loc[:, :"oth"])# , total=mineralogy['sum'])

In [17]:
# Renew 'sum' column to reflect changes applied during cleaning
mineralogy["sum"] = mineralogy.loc[:, :"oth"].sum(axis=1)

In [18]:
# Check that sum of all variables + 'sum' == 200
assert all(np.isclose(mineralogy.loc[:, :'sum'].sum(axis=1), 200.0))

In [19]:
mineralogy.to_excel("../_INTERPOLATION/normalised_values.xlsx")

### centred log-ratio (clr) transformation

In [20]:
mineralogy_clr = preproc.clr(mineralogy)
mineralogy_clr.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,oth,sum,hs
1,4.519180,-3.091673,2.445661,-0.367093,0.669527,-4.477967,-0.789088,-0.470634,0.820350,1.405355,-0.922619,4.734670,-4.475670
2,4.534383,-2.150229,2.464892,-1.619600,-0.427462,-3.759667,-0.763934,-0.897466,0.985266,1.781597,-0.445481,4.754020,-4.456320
3,4.425654,-2.951104,2.336152,-1.787953,0.592749,-3.867395,-2.075636,-1.921485,-0.243054,1.861080,0.139938,4.650098,-1.159045
4,4.285378,-3.311140,2.173657,0.047498,-0.379946,-2.394849,-0.890771,-0.492741,1.218768,1.354655,-1.439337,4.519584,-4.690756
5,4.046324,-2.851128,2.000902,-0.771686,0.630112,-3.544275,-3.321132,-0.102256,0.093311,0.799530,0.326926,4.285753,-1.592383


### Principal Component Analysis (PCA)

In [21]:
mineralogy_pca = preproc.pca(mineralogy_clr)
preproc.pca_variance(mineralogy_pca)

7 PCA components  out of 13 components with variance sum 0.9535306717860614 needed for obtaining sum of variance > 0.95


array([3.59050577e-01, 2.60958082e-01, 1.29041804e-01, 6.59819203e-02,
       5.75172491e-02, 4.46332610e-02, 3.63477772e-02, 2.62125369e-02,
       1.48989319e-02, 4.34031836e-03, 9.77100841e-04, 4.04401875e-05,
       2.24837794e-32])

In [22]:
mineralogy_pca_df = preproc.create_pca_df(mineralogy_pca, mineralogy_clr)

In [23]:
mineralogy_pca_df

,PC01,PC02,PC03,PC04,PC05,PC06,PC07,PC08,PC09,PC10,PC11,PC12,PC13
1,-0.447174,-1.225068,0.116980,-1.118125,0.072895,0.968142,0.135201,-1.046815,-0.163684,0.405531,0.210871,0.012263,-4.978631e-16
2,-0.376621,-1.410760,0.977915,-0.596661,-0.400204,0.249458,0.797682,0.539827,-0.157654,0.334672,0.207434,0.038930,1.605954e-16
3,3.187993,-1.541733,1.059821,-0.101622,-1.098362,-0.327546,0.899679,-1.063429,-0.901138,0.906119,0.141421,0.049035,2.368786e-16
4,-0.733521,-1.464722,-0.690817,0.790791,-0.052407,1.647700,0.216261,0.043582,-0.113234,-0.046977,0.278490,0.029399,-6.753686e-16
5,2.656838,-0.964694,0.074225,0.373453,-1.419034,-0.564535,-1.424684,-1.158970,-0.447363,0.403525,0.278001,0.015713,3.613596e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4655,0.892870,2.326275,-1.147952,0.585820,-0.731866,0.744900,-0.265913,-0.424258,0.030811,-0.273406,-0.207447,0.065138,4.183668e-16
4656,-2.408156,2.450686,-0.553429,0.009524,0.068795,0.014798,-0.218064,-0.688277,0.071384,0.193791,-0.133836,0.084928,6.753052e-16
4657,1.106035,3.018540,-1.204171,0.246965,-0.323195,-0.017034,0.076226,-0.156054,-0.219544,0.018146,-0.119151,0.087319,-4.693849e-16
4658,1.357590,2.827987,-0.736605,-0.025258,-0.809447,-0.410685,-0.559875,-0.039578,0.624331,-0.042759,-0.275724,0.061361,-3.417382e-16


In [24]:
preproc.save_obj(mineralogy_pca_df, "mineralogy_pca_df")

____

## Coordinates

In [25]:
coordinates = pd.read_excel("../_DATA/full_with_coordinates.xlsx", index_col=0, usecols=[0, 1, 2, 3])

In [26]:
coordinates

,Latitude,Longitude,past_mer
1,"44°31'30.0""","138°37'30.0""",NaN
2,"54°12'10.0""","119°24'0.0""",NaN
3,"62°36'0.0""","155°36'0.0""",NaN
4,"61°35'0.0""","146°2'0.0""",NaN
5,"68°55'0.0""","164°24'0.0""",NaN
...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN
4656,"46°56'30.0""","137°5'3.0""",NaN
4657,"58°12'0.0""","138°12'0.0""",NaN
4658,"60°51'0.0""","147°31'0.0""",NaN


In [27]:
# Delete negative signs in "Longitude" column for dms2dec function to work properly
coordinates["Longitude"] = coordinates["Longitude"].str.replace("-", "")

In [28]:
# Include W in "Longitude" column
sum_ = 0

for index, row in coordinates.iterrows():
    if ("W" in str(row["past_mer"])) or ("w" in str(row["past_mer"])):
        coordinates.loc[index, "Longitude"] = row["Longitude"] + "W"
        
        sum_ += 1

In [29]:
# Check that all occurences of "W" or "w" are catched
assert sum_ == int(coordinates["past_mer"].value_counts())

### Convert from degrees to decimal format

In [30]:
coordinates["Y"] = coordinates.loc[:, "Latitude"].apply(cleaning.dms2dec)
coordinates["X"] = coordinates.loc[:, "Longitude"].apply(cleaning.dms2dec)

In [31]:
# Check
coordinates.loc[42, "X"]

149.38333333333333

In [32]:
coordinates

,Latitude,Longitude,past_mer,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525000,138.625000
2,"54°12'10.0""","119°24'0.0""",NaN,54.202778,119.400000
3,"62°36'0.0""","155°36'0.0""",NaN,62.600000,155.600000
4,"61°35'0.0""","146°2'0.0""",NaN,61.583333,146.033333
5,"68°55'0.0""","164°24'0.0""",NaN,68.916667,164.400000
...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.700000,164.383333
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941667,137.084167
4657,"58°12'0.0""","138°12'0.0""",NaN,58.200000,138.200000
4658,"60°51'0.0""","147°31'0.0""",NaN,60.850000,147.516667


coordinates = coordinates.rename({"Y" : "Latitude"}, axis=1)
coordinates = coordinates.rename({"X" : "Longitude"}, axis=1)


** plotting in Qgis does not work --> will look into it (something to do with qgis) **

### adding noise to coordinates

**Although the idea was to only add noise to those data points that have replicates, I think it's okay to add noise to all data points.**

In [33]:
coordinates_noise = pd.read_excel("../_INTERPOLATION/coordinates_decimal.xlsx", index_col=0)

In [34]:
coordinates_noise = coordinates_noise.rename({"Y": "Y_without_noise", "X": "X_without_noise"}, axis=1)

In [35]:
# coordinates_noise["random_value"] = np.random.random(size=len(coordinates_noise))
# coordinates_noise["mean"] = 0
# coordinates_noise["std"] = 0.00007

# You don't seem to use the 'random_value' anywhere so commented it
# Instead of assigning the mean and std to every row it is much faster to define it once
# and then use it when calling the random normal sample.
noise_mean = 0
noise_std = 0.00007

In [36]:
# You can set up your own 'pseudo' random number generator with np.random.RandomState(x)
# where x is the random seed that you can choose, I just chose 4343
# This way the samples from the normal distribution will be random but will be the same every time you
# rerun this cell, so that your results in the second notebook 'Interpolation' will also stay the same
# even after rerunning. Otherwise, at every run of the next cell, numpy will choose a new random seed.
# You may also find online that people say to just set the RandomState with np.random.seed = 4343
# but this will affect every random process you start anywhere and that's not what you want for now
pnrg = np.random.RandomState(4343)

In [37]:
# coordinates_noise["normal_distribution"] =  np.random.normal(noise_mean, noise_std)

# Would sample two random numbers so that the coordinates can change in all direction and not just NE or SW
coordinates_noise["noise_for_X"] =  pnrg.normal(noise_mean, noise_std, size=coordinates_noise.shape[0])
coordinates_noise["noise_for_Y"] =  pnrg.normal(noise_mean, noise_std, size=coordinates_noise.shape[0])

In [38]:
coordinates_noise["Y"] = coordinates_noise["Y_without_noise"] + coordinates_noise["noise_for_X"]
coordinates_noise["X"] = coordinates_noise["X_without_noise"] + coordinates_noise["noise_for_Y"]

In [39]:
# NEW
coordinates_noise

,Latitude,Longitude,past_mer,Y_without_noise,X_without_noise,noise_for_X,noise_for_Y,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525027,138.625027,0.000058,-0.000078,44.525085,138.624949
2,"54°12'10.0""","119°24'0.0""",NaN,54.202861,119.400083,0.000016,-0.000237,54.202877,119.399847
3,"62°36'0.0""","155°36'0.0""",NaN,62.600001,155.600001,0.000044,-0.000109,62.600044,155.599891
4,"61°35'0.0""","146°2'0.0""",NaN,61.583302,146.033302,0.000132,-0.000076,61.583433,146.033226
5,"68°55'0.0""","164°24'0.0""",NaN,68.916552,164.399886,-0.000019,-0.000077,68.916533,164.399808
...,...,...,...,...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.699978,164.383312,-0.000118,0.000014,66.699861,164.383326
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941677,137.084177,-0.000016,-0.000020,46.941661,137.084157
4657,"58°12'0.0""","138°12'0.0""",NaN,58.199973,138.199973,-0.000079,-0.000040,58.199894,138.199934
4658,"60°51'0.0""","147°31'0.0""",NaN,60.849992,147.516659,-0.000163,0.000135,60.849829,147.516794


In [40]:
# OLD
coordinates_noise

,Latitude,Longitude,past_mer,Y_without_noise,X_without_noise,noise_for_X,noise_for_Y,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525027,138.625027,0.000058,-0.000078,44.525085,138.624949
2,"54°12'10.0""","119°24'0.0""",NaN,54.202861,119.400083,0.000016,-0.000237,54.202877,119.399847
3,"62°36'0.0""","155°36'0.0""",NaN,62.600001,155.600001,0.000044,-0.000109,62.600044,155.599891
4,"61°35'0.0""","146°2'0.0""",NaN,61.583302,146.033302,0.000132,-0.000076,61.583433,146.033226
5,"68°55'0.0""","164°24'0.0""",NaN,68.916552,164.399886,-0.000019,-0.000077,68.916533,164.399808
...,...,...,...,...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.699978,164.383312,-0.000118,0.000014,66.699861,164.383326
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941677,137.084177,-0.000016,-0.000020,46.941661,137.084157
4657,"58°12'0.0""","138°12'0.0""",NaN,58.199973,138.199973,-0.000079,-0.000040,58.199894,138.199934
4658,"60°51'0.0""","147°31'0.0""",NaN,60.849992,147.516659,-0.000163,0.000135,60.849829,147.516794


In [41]:
coordinates_noise.to_excel("../_DATA/full_with_coordinates_noice_final_new.xlsx")

In [42]:
coordinates = pd.read_excel("../_DATA/full_with_coordinates_noice_final.xlsx", index_col=0, usecols=[0, 1, 2, 3, 10, 11])

In [43]:
coordinates

,Latitude,Longitude,past_mer,Y,X
1,"44°31'30.0""","138°37'30.0""",NaN,44.525040,138.625040
2,"54°12'10.0""","119°24'0.0""",NaN,54.202798,119.400020
3,"62°36'0.0""","155°36'0.0""",NaN,62.599978,155.599978
4,"61°35'0.0""","146°2'0.0""",NaN,61.583249,146.033249
5,"68°55'0.0""","164°24'0.0""",NaN,68.916572,164.399905
...,...,...,...,...,...
4655,"66°42'0.0""","164°23'0.0""",NaN,66.699992,164.383326
4656,"46°56'30.0""","137°5'3.0""",NaN,46.941584,137.084084
4657,"58°12'0.0""","138°12'0.0""",NaN,58.200076,138.200076
4658,"60°51'0.0""","147°31'0.0""",NaN,60.849958,147.516625


### Convert to UTM coordinates

**To do**
* Group samples into certain groups based on spatial distribution
* Recalculate utm coordinates based on fixed zone (fixed letter and number)

In [44]:
coordinates_utm = coordinates.apply(lambda row : utm.from_latlon(row["Y"], row["X"], force_zone_number=48, force_zone_letter='T'), axis=1)
coordinates_utm = coordinates_utm.apply(pd.Series)
coordinates_utm.columns = ["Y_UTM", "X_UTM", "ZoneNumber", "ZoneLetter"]

In [45]:
coordinates_utm

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter
1,3.166850e+06,5.512837e+06,48,T
2,1.436111e+06,6.102354e+06,48,T
3,2.874853e+06,7.966637e+06,48,T
4,2.565651e+06,7.521202e+06,48,T
5,2.540731e+06,8.757731e+06,48,T
...,...,...,...,...
4655,2.759727e+06,8.616851e+06,48,T
4656,2.927123e+06,5.721711e+06,48,T
4657,2.397980e+06,6.939549e+06,48,T
4658,2.684981e+06,7.505261e+06,48,T


In [46]:
coordinates_utm["ZoneNumber"].value_counts()

48    4659
Name: ZoneNumber, dtype: int64

In [47]:
coordinates_utm["ZoneLetter"].value_counts()

T    4659
Name: ZoneLetter, dtype: int64

In [48]:
(coordinates_utm["ZoneNumber"].astype(str) + coordinates_utm["ZoneLetter"]).value_counts()

48T    4659
dtype: int64

In [49]:
coordinates_utm.to_excel("../_INTERPOLATION/coordinates_UTM.xlsx")
cof= pd.read_excel("../_RESULTS/working_data.xlsx", index_col=0, usecols = lambda column : column not in ["Lat_deg", "Lat_min", "Lat_sec", "Long_deg", "Long_min", "Long_sec", "past_mer"] )

In [50]:
cof

,type_granite,time,massif,sampler,others,sampler+year
512,Granite coarse-grained,Tr-J,Yugalkan massif,T.A.Alfer'eva,NaN,"A.D.Kanischev,1959"
1299,Granite coarse-grained,Tr-J,NaN,G.L.Znamenskaya,NaN,"N.P.Kostyakov,1961"
1536,Bt granite porphyraceous,Tr-J,NaN,E.G.Ivanova,NaN,"L.A.Kozubova,1957"
1760,Bt granite medium-grained,Tr-J,NaN,NaN,NaN,"V.I.Fel'dman,1956"
1890,Plagiogranite,Tr-J,NaN,NI.Serebryakova,NaN,"A.V.Vnukov,1959"
...,...,...,...,...,...,...
1150,Granite leucocratic,J,Upper-Buy massif,NaN,.N.P.Mel'nikova,"V.V.Starchenko,1968"
1396,Granite,Tr,Ust'-Nerchugan massifmassif,N.Aolebedeva,NaN,"K.F.Khatskevich,1967"
2116,Granite,NaN,NaN,N.I.Serebryakov~,NaN,"V.Yu.Shenfil,1962"
2973,Bt granite-porphyry,NaN,Ergelyakh massif,G.P.Ignatovich,NaN,"G.G.Naumov,1978"


In [51]:
coordinates_full = pd.concat([coordinates_utm, coordinates, mineralogy, cof], axis = 1)

In [52]:
coordinates_full.to_excel("../_INTERPOLATION/coordinates_full_data.xlsx")


### grouping the data

In [53]:
areax = coordinates_full
areax["area"] = ""


In [54]:
areay1 = areax[areax["X"].between(107.20, 121)]
area1 = areay1[areay1["Y"].between(48.85, 56.6)]
area1["area"] = 1
area1.to_excel("../_INTERPOLATION/area1.xlsx")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [55]:
areay2 = areax[areax["X"].between(131.75, 141.5)]
area2 = areay2[areay2["Y"].between(42, 56)]
area2["area"] = 2
area2.to_excel("../_INTERPOLATION/area2.xlsx")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [56]:
areay3 = areax[areax["X"].between(133, 160.5)]
area3 = areay3[areay3["Y"].between(58.6, 70.85)]
area3["area"] = 3
area3.to_excel("../_INTERPOLATION/area3.xlsx")

C:\Users\Sebastian\Anaconda3\envs\main\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
areay4 = areax[areax["X"].between(160.5, 180)]
area4 = areay4[areay4["Y"].between(58, 75)]
area4["area"] = 4
area4.to_excel("../_INTERPOLATION/area4.xlsx")

In [58]:
areay5 = areax[areax["X"].between(-180, -165)]
area5 = areay5[areay5["Y"].between(63, 70)]
area5["area"] = 5
area5.to_excel("../_INTERPOLATION/area5.xlsx")

In [59]:
area12 = area1.append(area2)
area123 = area12.append(area3) 
area1234 = area123.append(area4) 
area = area1234.append(area5) 
area.to_excel("../_INTERPOLATION/area_subdivided.xlsx")

____

In [60]:
area1_J = area1.loc[area1['time'] == 'J']
area1_Tr = area1.loc[area1['time'] == 'Tr']
area1_Tr_J = area1.loc[area1['time'] == 'Tr-J']

area1_J.to_excel("../_INTERPOLATION/area1_J.xlsx")
area1_Tr.to_excel("../_INTERPOLATION/area1_Tr.xlsx")
area1_Tr_J.to_excel("../_INTERPOLATION/area1_Tr_J.xlsx")

## Metadata

In [61]:
metadata = pd.read_excel("../_INTERPOLATION/coordinates_full_data.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])

In [62]:
metadata

,type_granite,time,massif,sampler,others,sampler+year
1,Granite leucogranitic,K,NaN,NaN,NaN,"V.N.Musin,1970"
2,Granite leucocratic,Tr,NaN,V.I.Zhigalova,NaN,"E.A.Ivanov,1969"
3,Granite leucocratic coarse-grained,K,Omsukchan massif,P.M.Bosek,NaN,"O.S.Gracheva,1948"
4,Granite leucocratic,K,Buksandzhin massif,A.Kh.Brovtman,NaN,"A.F.Mikhaylov,1948"
5,Granite-porphyry micropegmatitic,K,Attykveem massif,L.G.Semenova,0th.:S-0.16,"A.I.Sadovsky,1963"
...,...,...,...,...,...,...
4655,Diorite,K,Egdegkych massif,NaN,Oth.:co2-0.12,"V.A.lgnat'ev,1964"
4656,Quartz diorite,K,Verkhneplotnikovsky massif,NaN,NaN,"A.A.Syas'ko,1969"
4657,Diorite,J,NaN,NaN,"Oth.:co2-0.02,so3-0.0l","N.N.Remizov,1967"
4658,Diorite,K,NaN,NaN,NaN,"A.P.Osipov,1966"


In [63]:
metadata["type_granite"].value_counts()

Granite                                 656
Granodiorite                            523
Granite-porphyry                        282
Bt granite                              268
Granite leucocratic                     220
                                       ... 
Granite pegmatoid alkaline miarolic       1
Diorite leucocratic                       1
Augen diorite                             1
Two mica granite uneven-grained           1
Granodiorite(granite) medium-grained      1
Name: type_granite, Length: 401, dtype: int64

In [64]:
metadata["massif"].value_counts()

Ulakhan-Sis massif         58
Khoboyotuu-Echiy massif    56
Vladimirsky massif         35
Zimov'e massif             33
Bom-Gorkhon massif         31
                           ..
Supkan'ya massif            1
Shukumin massif             1
Sylarbin massif             1
Lower-Kintsukhin massif     1
Lower-Ramsky massif         1
Name: massif, Length: 941, dtype: int64

In [65]:
metadata["time"].value_counts()

K       2522
J        825
Tr       517
Pg       365
Tr-J     338
Mz        88
Name: time, dtype: int64

In [66]:
metadata["sampler"].value_counts()

L.S.Voronova       120
D.M.Shuster         87
N.A.Lebedeva        86
V.I.Zhigalova       70
N.P.Mel'nikova      51
                  ... 
G.A.Filippova        1
EI.Chechukalova      1
T.V.Chernykh         1
AN.Geraskina         1
O.GArbidan           1
Name: sampler, Length: 736, dtype: int64

In [67]:
metadata["sampler+year"].value_counts()

G.A.Valuy,1975        76
V.A.Popeko,1968       61
V.A.Faradzhev,1971    43
V.S.Ivanov,1968       37
R.O.Galabala,1976     37
                      ..
D.S.Golota,1945        1
M.D.Elianov,1953       1
S.V.Bravina'1955       1
Yaskelyainen,1967      1
V.D.Volodin,1953       1
Name: sampler+year, Length: 1392, dtype: int64

## data for each area

In [68]:
metadata_area1 = pd.read_excel("../_INTERPOLATION/area1.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area2 = pd.read_excel("../_INTERPOLATION/area2.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area3 = pd.read_excel("../_INTERPOLATION/area3.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area4 = pd.read_excel("../_INTERPOLATION/area4.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area5 = pd.read_excel("../_INTERPOLATION/area5.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])

metadata_area1_J = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area1_Tr_J = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])
metadata_area1_Tr = pd.read_excel("../_INTERPOLATION/area1_Tr.xlsx", index_col=0, usecols=[0, 23, 24, 25, 26, 27, 28])

In [69]:
coordinates_utm_area1 = pd.read_excel("../_INTERPOLATION/area1.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area2 = pd.read_excel("../_INTERPOLATION/area2.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area3 = pd.read_excel("../_INTERPOLATION/area3.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area4 = pd.read_excel("../_INTERPOLATION/area4.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area5 = pd.read_excel("../_INTERPOLATION/area5.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])

coordinates_utm_area1_J = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area1_Tr_J = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])
coordinates_utm_area1_Tr = pd.read_excel("../_INTERPOLATION/area1_Tr.xlsx", index_col=0, usecols=[0, 1, 2, 3, 4])

In [70]:
coordinates_area1 = pd.read_excel("../_INTERPOLATION/area1.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area2 = pd.read_excel("../_INTERPOLATION/area2.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area3 = pd.read_excel("../_INTERPOLATION/area3.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area4 = pd.read_excel("../_INTERPOLATION/area4.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area5 = pd.read_excel("../_INTERPOLATION/area5.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])

coordinates_area1_J = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area1_Tr_J = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])
coordinates_area1_Tr = pd.read_excel("../_INTERPOLATION/area1_Tr.xlsx", index_col=0, usecols=[0, 5, 6, 7, 8, 9])

In [71]:
mineralogy_area1 = pd.read_excel("../_INTERPOLATION/area1.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area2 = pd.read_excel("../_INTERPOLATION/area2.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area3 = pd.read_excel("../_INTERPOLATION/area3.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area4 = pd.read_excel("../_INTERPOLATION/area4.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area5 = pd.read_excel("../_INTERPOLATION/area5.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

mineralogy_area1_J = pd.read_excel("../_INTERPOLATION/area1_J.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area1_Tr_J = pd.read_excel("../_INTERPOLATION/area1_Tr_J.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
mineralogy_area1_Tr = pd.read_excel("../_INTERPOLATION/area1_Tr.xlsx", index_col=0, usecols=[0, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [72]:
mineralogy_area1

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,oth
2,80.280983,0.100351,10.135474,0.170597,0.561967,0.020070,0.401405,0.351229,2.308078,5.117913,0.551932
6,78.907496,0.219298,10.915072,0.119617,1.694577,0.029904,0.159490,0.608054,2.990431,3.907496,0.448565
9,79.063693,0.080370,11.271851,0.381756,1.446655,0.020092,0.653004,0.331525,2.511553,3.918023,0.321479
10,78.247885,0.099552,11.946242,0.179194,0.746640,0.019910,0.079642,0.408163,2.767546,4.678945,0.826282
11,78.288621,0.149596,11.000299,0.418869,1.316446,0.029919,0.159569,0.578438,3.121572,4.428044,0.508627
...,...,...,...,...,...,...,...,...,...,...,...
4538,57.821425,0.701473,16.875438,2.715703,4.108628,0.050105,4.068544,5.942479,4.609680,2.505261,0.601263
4552,57.060347,1.125274,16.819359,3.166700,4.371639,0.069707,3.774149,6.273651,3.774149,2.698666,0.866361
4574,56.693071,0.842274,16.865537,3.178582,3.599719,0.030081,4.391858,7.089141,4.462048,2.205956,0.641733
4585,56.189235,1.302997,18.522602,2.345394,4.620627,0.110254,2.956801,5.312218,4.981457,2.555879,1.102536


In [73]:
mineralogy_clr_area1 = preproc.clr(mineralogy_area1)
mineralogy_clr_area2 = preproc.clr(mineralogy_area2)
mineralogy_clr_area3 = preproc.clr(mineralogy_area3)
mineralogy_clr_area4 = preproc.clr(mineralogy_area4)
mineralogy_clr_area5 = preproc.clr(mineralogy_area5)

mineralogy_clr_area1_J = preproc.clr(mineralogy_area1_J)
mineralogy_clr_area1_Tr_J = preproc.clr(mineralogy_area1_Tr_J)
mineralogy_clr_area1_Tr = preproc.clr(mineralogy_area1_Tr)

In [74]:
mineralogy_pca_area1 = preproc.pca(mineralogy_clr_area1)
preproc.pca_variance(mineralogy_pca_area1)

mineralogy_pca_area2 = preproc.pca(mineralogy_clr_area2)
preproc.pca_variance(mineralogy_pca_area2)

mineralogy_pca_area3 = preproc.pca(mineralogy_clr_area3)
preproc.pca_variance(mineralogy_pca_area3)

mineralogy_pca_area4 = preproc.pca(mineralogy_clr_area4)
preproc.pca_variance(mineralogy_pca_area4)

mineralogy_pca_area5 = preproc.pca(mineralogy_clr_area5)
preproc.pca_variance(mineralogy_pca_area5)



mineralogy_pca_area1_J = preproc.pca(mineralogy_clr_area1_J)
preproc.pca_variance(mineralogy_pca_area1_J)

mineralogy_pca_area1_Tr_J = preproc.pca(mineralogy_clr_area1_Tr_J)
preproc.pca_variance(mineralogy_pca_area1_Tr_J)

mineralogy_pca_area1_Tr = preproc.pca(mineralogy_clr_area1_Tr)
preproc.pca_variance(mineralogy_pca_area1_Tr)

7 PCA components  out of 11 components with variance sum 0.9782347091275596 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.97528158450117 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.9762810063432605 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.959185389967746 needed for obtaining sum of variance > 0.95
6 PCA components  out of 11 components with variance sum 0.9539561185088523 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.9743034179578702 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.9861295691887149 needed for obtaining sum of variance > 0.95
7 PCA components  out of 11 components with variance sum 0.9797427087025801 needed for obtaining sum of variance > 0.95


array([3.63451512e-01, 2.12810591e-01, 1.16723778e-01, 1.04719394e-01,
       8.26674196e-02, 5.82383733e-02, 4.11316412e-02, 1.42036345e-02,
       4.22045683e-03, 1.83319992e-03, 3.03620722e-32])

In [75]:
mineralogy_pca_area1_df = preproc.create_pca_df(mineralogy_pca_area1, mineralogy_clr_area1)
mineralogy_pca_area2_df = preproc.create_pca_df(mineralogy_pca_area2, mineralogy_clr_area2)
mineralogy_pca_area3_df = preproc.create_pca_df(mineralogy_pca_area3, mineralogy_clr_area3)
mineralogy_pca_area4_df = preproc.create_pca_df(mineralogy_pca_area4, mineralogy_clr_area4)
mineralogy_pca_area5_df = preproc.create_pca_df(mineralogy_pca_area5, mineralogy_clr_area5)


mineralogy_pca_area1_J_df = preproc.create_pca_df(mineralogy_pca_area1_J, mineralogy_clr_area1_J)
mineralogy_pca_area1_Tr_J_df = preproc.create_pca_df(mineralogy_pca_area1_Tr_J, mineralogy_clr_area1_Tr_J)
mineralogy_pca_area1_Tr_df = preproc.create_pca_df(mineralogy_pca_area1_Tr, mineralogy_clr_area1_Tr)


## Saving of data

In [76]:
# Save data as pickle files to use them in later notebooks
preproc.save_obj(mineralogy_area1, "mineralogy_area1") # mineralogy
preproc.save_obj(mineralogy_clr_area1, "mineralogy_clr_area1") # mineralogy clr
preproc.save_obj(mineralogy_pca_area1, "mineralogy_pca_area1") # mineralogy pca info
preproc.save_obj(mineralogy_pca_area1_df, "mineralogy_pca_area1_df") # mineralogy pca scores

preproc.save_obj(coordinates_area1, "coordinates_area1") # coordinates latlon
preproc.save_obj(coordinates_utm_area1, "coordinates_utm_area1") # coordinates utm
preproc.save_obj(metadata_area1, "metadata_area1") # metadata


preproc.save_obj(mineralogy_area1_J, "mineralogy_area1_J") # mineralogy
preproc.save_obj(mineralogy_clr_area1_J, "mineralogy_clr_area1_J") # mineralogy clr
preproc.save_obj(mineralogy_pca_area1_J, "mineralogy_pca_area1_J") # mineralogy pca info
preproc.save_obj(mineralogy_pca_area1_J_df, "mineralogy_pca_area1_J_df") # mineralogy pca scores

preproc.save_obj(coordinates_area1_J, "coordinates_area1_J") # coordinates latlon
preproc.save_obj(coordinates_utm_area1_J, "coordinates_utm_area1_J") # coordinates utm
preproc.save_obj(metadata_area1_J, "metadata_area1_J") # metadata


preproc.save_obj(mineralogy_area1_Tr_J, "mineralogy_area1_Tr_J") # mineralogy
preproc.save_obj(mineralogy_clr_area1_Tr_J, "mineralogy_clr_area1_Tr_J") # mineralogy clr
preproc.save_obj(mineralogy_pca_area1_Tr_J, "mineralogy_pca_area1_Tr_J") # mineralogy pca info
preproc.save_obj(mineralogy_pca_area1_Tr_J_df, "mineralogy_pca_area1_Tr_J_df") # mineralogy pca scores

preproc.save_obj(coordinates_area1_Tr_J, "coordinates_area1_Tr_J") # coordinates latlon
preproc.save_obj(coordinates_utm_area1_Tr_J, "coordinates_utm_area1_Tr_J") # coordinates utm
preproc.save_obj(metadata_area1_Tr_J, "metadata_area1_Tr_J") # metadata


preproc.save_obj(mineralogy_area1_Tr, "mineralogy_area1_Tr") # mineralogy
preproc.save_obj(mineralogy_clr_area1_Tr, "mineralogy_clr_area1_Tr") # mineralogy clr
preproc.save_obj(mineralogy_pca_area1_Tr, "mineralogy_pca_area1_Tr") # mineralogy pca info
preproc.save_obj(mineralogy_pca_area1_Tr_df, "mineralogy_pca_area1_Tr_df") # mineralogy pca scores

preproc.save_obj(coordinates_area1_Tr, "coordinates_area1_Tr") # coordinates latlon
preproc.save_obj(coordinates_utm_area1_Tr, "coordinates_utm_area1_Tr") # coordinates utm
preproc.save_obj(metadata_area1_Tr, "metadata_area1_Tr") # metadata




____

In [77]:
coordinates_utm_area1

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter
2,1.436111e+06,6.102354e+06,48,T
6,1.113927e+06,5.652652e+06,48,T
9,1.068636e+06,5.598845e+06,48,T
10,1.098840e+06,5.690263e+06,48,T
11,9.048985e+05,5.630297e+06,48,T
...,...,...,...,...
4538,1.304961e+06,5.902228e+06,48,T
4552,1.328849e+06,6.028456e+06,48,T
4574,1.413275e+06,6.019949e+06,48,T
4585,1.455811e+06,5.990438e+06,48,T


In [78]:
mineralogy_clr_area1

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,oth
2,4.561447,-2.123165,2.491956,-1.592537,-0.400398,-3.732603,-0.736871,-0.870402,1.012329,1.808661,-0.418417
6,4.417489,-1.468110,2.439357,-2.074246,0.576646,-3.460540,-1.786564,-0.448279,1.144630,1.412109,-0.752490
9,4.425549,-2.465823,2.477604,-0.907678,0.424549,-3.852117,-0.370877,-1.048757,0.976197,1.420882,-1.079528
10,4.582686,-2.084271,2.703221,-1.496485,-0.069368,-3.693709,-2.307415,-0.673284,1.240765,1.765876,0.031984
11,4.331171,-1.929048,2.368691,-0.899428,0.245704,-3.538486,-1.864509,-0.576655,1.109105,1.458727,-0.705272
...,...,...,...,...,...,...,...,...,...,...,...
4538,3.050960,-1.360972,1.819460,-0.007349,0.406690,-4.000030,0.396886,0.775727,0.521759,-0.088007,-1.515123
4552,2.926741,-0.999342,1.705162,0.035321,0.357769,-3.780820,0.210806,0.718990,0.210806,-0.124611,-1.260822
4574,3.046178,-1.163124,1.833798,0.164961,0.289382,-4.495328,0.488278,0.967090,0.504134,-0.200313,-1.435057
4585,2.866339,-0.897719,1.756606,-0.309932,0.368145,-3.367358,-0.078278,0.507624,0.443337,-0.223990,-1.064773
